In [5]:
import torch
import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, file_path):
        df = pd.read_csv(file_path)
        self.x = df.iloc[:, 0].values
        self.y = df.iloc[:, 1].values
        self.length = len(df)


    def __getitem__(self, index):
        x = torch.FloatTensor([self.x[index] ** 2, self.x[index]])
        y = torch.FloatTensor([self.y[index]])

        return x, y


    def __len__(self):
        return self.length

class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.layer = nn.Linear(2, 1)


    def forward(self, x):
        x = self.layer(x)

        return x

train_dataset = CustomDataset("./dataset.csv")
train_dataLoader = DataLoader(train_dataset, batch_size = 128, drop_last = True)

device = "cuda" if  torch.cuda.is_available() else "cpu"
model = CustomModel().to(device)
criterion = nn.MSELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr = 0.00001)


checkpoint = 1

for epoch in range(10000):
    cost = 0.0


    for x, y in train_dataLoader:
        x = x.to(device)
        y = y.to(device)

        output = model(x)
        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optim.step()

        cost += loss

    cost = cost / len(train_dataset)

    if (epoch + 1) % 1000 == 0:
        torch.save(
                {
                    "model" : "CustomModel",
                    "epoch" : epoch,
                    "model_state_dict" : model.state_dict(),
                    "optimizer_satt_dict" : optimizer.state_dict(), 
                    "cost" : cost,
                    "description" : f"CustomModel 체크포인트-{checkpoint}",
                },
                f"./checkpoint-{checkpoint}.pt",
         )
        checkpoint += 1    


In [7]:
from tensorflow import keras
import tensorflow as tf
import matplotlib.pyplot as plt

hist = model().fit(X_train, Y_train, epochs=10000, batch_size=10, validation_data=(X_val, Y_val))

fig, loss_ax = plt.subplots()

acc_ax = loss_ax.twinx()

loss_ax.plot(hist.history['loss'], 'y', label='train loss')
loas_ax.plot(hist.history['val_loss'], 'r', label='val loss')

acc_ax.plot(hist.history['acc'], 'b', label='train acc')
acc_ax.plot(hist.history['val_acc'], 'g', label='val acc')

loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
acc_ax.set_ylabel('accuray')

loss_ax.legend(loc='upper left')
acc_ax.legend(loc='lower left')

plt.show()


TypeError: CustomModel.forward() missing 1 required positional argument: 'x'